## Data import

In [101]:
# import necessary packages for case study

import pandas as pd
import numpy as np
import statistics
import statsmodels.api as sm

### Load data

In [102]:
df_fundamentals = pd.read_csv("case_business-valuation_data/case3_fundamentals.csv", index_col=0) # to avoid double index entries
df_price_split = pd.read_csv("case_business-valuation_data/case3_prices-split-adjusted.csv")
df_securities = pd.read_csv("case_business-valuation_data/case3_securities.csv")

In [103]:
# Customize the names of the records by replacing ' ' with '.'

df_fundamentals.columns = [c.replace(' ', '.') for c in df_fundamentals.columns]
df_price_split.columns = [c.replace(' ', '.') for c in df_price_split.columns]
df_securities.columns = [c.replace(' ', '.') for c in df_securities.columns]

In [104]:
df_fundamentals.columns

Index(['Ticker.Symbol', 'Period.Ending', 'Accounts.Payable',
       'Accounts.Receivable', 'Add'l.income/expense.items', 'After.Tax.ROE',
       'Capital.Expenditures', 'Capital.Surplus', 'Cash.Ratio',
       'Cash.and.Cash.Equivalents', 'Changes.in.Inventories', 'Common.Stocks',
       'Cost.of.Revenue', 'Current.Ratio', 'Deferred.Asset.Charges',
       'Deferred.Liability.Charges', 'Depreciation',
       'Earnings.Before.Interest.and.Tax', 'Earnings.Before.Tax',
       'Effect.of.Exchange.Rate',
       'Equity.Earnings/Loss.Unconsolidated.Subsidiary', 'Fixed.Assets',
       'Goodwill', 'Gross.Margin', 'Gross.Profit', 'Income.Tax',
       'Intangible.Assets', 'Interest.Expense', 'Inventory', 'Investments',
       'Liabilities', 'Long-Term.Debt', 'Long-Term.Investments',
       'Minority.Interest', 'Misc..Stocks', 'Net.Borrowings', 'Net.Cash.Flow',
       'Net.Cash.Flow-Operating', 'Net.Cash.Flows-Financing',
       'Net.Cash.Flows-Investing', 'Net.Income', 'Net.Income.Adjustments',
  

## Data preparation

### Converting data types

An overview of the content and structure of the imported data shows that some attributes are assigned the wrong data type.

In [105]:
df_price_split.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    851264 non-null  object 
 1   symbol  851264 non-null  object 
 2   open    851264 non-null  float64
 3   close   851264 non-null  float64
 4   low     851264 non-null  float64
 5   high    851264 non-null  float64
 6   volume  851264 non-null  float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [106]:
df_price_split['date'] = pd.to_datetime(df_price_split['date'], format='%Y-%m-%d')
df_price_split['date']

0        2016-01-05
1        2016-01-06
2        2016-01-07
3        2016-01-08
4        2016-01-11
            ...    
851259   2016-12-30
851260   2016-12-30
851261   2016-12-30
851262   2016-12-30
851263   2016-12-30
Name: date, Length: 851264, dtype: datetime64[ns]

In [107]:
df_securities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Ticker.symbol            505 non-null    object
 1   Security                 505 non-null    object
 2   SEC.filings              505 non-null    object
 3   GICS.Sector              505 non-null    object
 4   GICS.Sub.Industry        505 non-null    object
 5   Address.of.Headquarters  505 non-null    object
 6   Date.first.added         307 non-null    object
 7   CIK                      505 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 31.7+ KB


In [108]:
df_securities['Date.first.added'] = pd.to_datetime(df_securities['Date.first.added'], format='%Y-%m-%d')
df_securities['CIK'] = df_securities.CIK.astype('category')
df_securities['Security'] = df_securities.Security.astype('category')
df_securities['GICS.Sub.Industry'] = df_securities['GICS.Sub.Industry'].astype('category')
df_securities['GICS.Sector'] = df_securities['GICS.Sector'].astype('category')
df_securities['Address.of.Headquarters'] = df_securities['Address.of.Headquarters'].astype('category')
df_securities['SEC.filings'] = df_securities['SEC.filings'].astype('category')

In [109]:
df_securities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Ticker.symbol            505 non-null    object        
 1   Security                 505 non-null    category      
 2   SEC.filings              505 non-null    category      
 3   GICS.Sector              505 non-null    category      
 4   GICS.Sub.Industry        505 non-null    category      
 5   Address.of.Headquarters  505 non-null    category      
 6   Date.first.added         307 non-null    datetime64[ns]
 7   CIK                      505 non-null    category      
dtypes: category(6), datetime64[ns](1), object(1)
memory usage: 68.3+ KB


In [110]:
df_fundamentals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1781 entries, 0 to 1780
Data columns (total 78 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Ticker.Symbol                                        1781 non-null   object 
 1   Period.Ending                                        1781 non-null   object 
 2   Accounts.Payable                                     1781 non-null   float64
 3   Accounts.Receivable                                  1781 non-null   float64
 4   Add'l.income/expense.items                           1781 non-null   float64
 5   After.Tax.ROE                                        1781 non-null   float64
 6   Capital.Expenditures                                 1781 non-null   float64
 7   Capital.Surplus                                      1781 non-null   float64
 8   Cash.Ratio                                           1482 non-null   floa

In [111]:
df_fundamentals['Period.Ending'] = pd.to_datetime(df_fundamentals['Period.Ending'], format='%Y-%m-%d')
df_fundamentals['Period.Ending']

0      2012-12-31
1      2013-12-31
2      2014-12-31
3      2015-12-31
4      2012-12-29
          ...    
1776   2015-12-31
1777   2013-12-31
1778   2014-12-31
1779   2015-12-31
1780   2016-12-31
Name: Period.Ending, Length: 1781, dtype: datetime64[ns]

### Merging data

In [112]:
df_merge1 = df_fundamentals.merge(df_price_split, left_on=['Period.Ending', 'Ticker.Symbol'], right_on=['date', 'symbol'])
df_merge1 = df_merge1.drop(['date', 'symbol'], axis=1)

df_merge1

,Ticker.Symbol,Period.Ending,Accounts.Payable,Accounts.Receivable,Add'l.income/expense.items,After.Tax.ROE,Capital.Expenditures,Capital.Surplus,Cash.Ratio,Cash.and.Cash.Equivalents,...,Total.Revenue,Treasury.Stock,For.Year,Earnings.Per.Share,Estimated.Shares.Outstanding,open,close,low,high,volume
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08,12.850000,13.500000,12.840000,13.680000,7005600.0
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08,24.740000,25.250000,24.629999,25.250000,7166600.0
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,4.265000e+10,0.0,2014.0,4.02,7.169154e+08,53.900002,53.630001,53.320000,54.639999,10626000.0
3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,4.099000e+10,0.0,2015.0,11.39,6.681299e+08,42.540001,42.349998,41.830002,42.570000,6788900.0
4,ABBV,2013-12-31,6.448000e+09,681000000.0,-5.400000e+07,92.0,-4.910000e+08,3.671000e+09,144.0,9.595000e+09,...,1.879000e+10,-320000000.0,2013.0,2.58,1.600000e+09,52.990002,52.810001,52.360001,53.060001,3019700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,ZION,2014-12-31,0.000000e+00,0.0,-4.442200e+07,5.0,-1.757990e+08,0.000000e+00,NaN,1.396843e+10,...,2.361631e+09,0.0,2014.0,1.68,2.371798e+08,28.680000,28.510000,28.490000,28.770000,1470800.0
1353,ZION,2015-12-31,0.000000e+00,0.0,-2.530000e+06,4.0,-1.573610e+08,0.000000e+00,NaN,1.576313e+10,...,2.210591e+09,0.0,2015.0,1.20,2.578925e+08,27.260000,27.299999,27.139999,27.639999,1737600.0
1354,ZTS,2013-12-31,1.381000e+09,-99000000.0,9.000000e+06,54.0,-1.840000e+08,8.780000e+08,43.0,6.100000e+08,...,4.561000e+09,0.0,2013.0,1.01,4.990099e+08,32.799999,32.689999,32.459999,32.810001,2270400.0
1355,ZTS,2014-12-31,1.071000e+09,69000000.0,-7.000000e+06,44.0,-1.800000e+08,9.580000e+08,81.0,8.820000e+08,...,4.785000e+09,0.0,2014.0,1.16,5.025862e+08,43.459999,43.029999,43.000000,43.880001,2131900.0


In [113]:
df_merge2 = df_merge1.merge(df_securities, left_on='Ticker.Symbol', right_on='Ticker.symbol')
df_merge2 = df_merge2.drop('Ticker.symbol', axis=1)

df_merge2

,Ticker.Symbol,Period.Ending,Accounts.Payable,Accounts.Receivable,Add'l.income/expense.items,After.Tax.ROE,Capital.Expenditures,Capital.Surplus,Cash.Ratio,Cash.and.Cash.Equivalents,...,low,high,volume,Security,SEC.filings,GICS.Sector,GICS.Sub.Industry,Address.of.Headquarters,Date.first.added,CIK
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,12.840000,13.680000,7005600.0,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,24.629999,25.250000,7166600.0,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,53.320000,54.639999,10626000.0,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201
3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,41.830002,42.570000,6788900.0,American Airlines Group,reports,Industrials,Airlines,"Fort Worth, Texas",2015-03-23,6201
4,ABBV,2013-12-31,6.448000e+09,681000000.0,-5.400000e+07,92.0,-4.910000e+08,3.671000e+09,144.0,9.595000e+09,...,52.360001,53.060001,3019700.0,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,ZION,2014-12-31,0.000000e+00,0.0,-4.442200e+07,5.0,-1.757990e+08,0.000000e+00,NaN,1.396843e+10,...,28.490000,28.770000,1470800.0,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",NaT,109380
1353,ZION,2015-12-31,0.000000e+00,0.0,-2.530000e+06,4.0,-1.573610e+08,0.000000e+00,NaN,1.576313e+10,...,27.139999,27.639999,1737600.0,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",NaT,109380
1354,ZTS,2013-12-31,1.381000e+09,-99000000.0,9.000000e+06,54.0,-1.840000e+08,8.780000e+08,43.0,6.100000e+08,...,32.459999,32.810001,2270400.0,Zoetis,reports,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21,1555280
1355,ZTS,2014-12-31,1.071000e+09,69000000.0,-7.000000e+06,44.0,-1.800000e+08,9.580000e+08,81.0,8.820000e+08,...,43.000000,43.880001,2131900.0,Zoetis,reports,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21,1555280


### Adjusting the data

The multiple is generally sector-specific. As the financial sector (banks, insurance companies and other financial institutions) differs fundamentally from other sectors in terms of company valuation using multiples, observations from this sector are excluded from further consideration.

In [114]:
# Exclude the sector 'Financials' as it generally behaves different as compared to the other sectors.
df_merge2 = df_merge2[df_merge2['GICS.Sector'] != 'Financials']

Observations lacking values in the column of estimated shares outstanding `Estimated.Shares.Outstanding` and observations with implausible negative values are also eliminated.

In [115]:
# Remove rows containing NA
df_merge2 = df_merge2.dropna(subset=['Estimated.Shares.Outstanding'])

#Remove entries that are not reasonable
df_merge2 = df_merge2[df_merge2['Earnings.Before.Tax']>0]
df_merge2 = df_merge2[df_merge2['Short-Term.Debt./.Current.Portion.of.Long-Term.Debt']>0]
df_merge2 = df_merge2[df_merge2['Estimated.Shares.Outstanding']>0]
df_merge2 = df_merge2[df_merge2['Sales,.General.and.Admin.']>0]

Finally, annual closing data are filtered and the date of the variable `Period.Ending` is reduced to its annual value.

In [116]:
# Filter 'Period.Ending' for the last day of each year

df_ana12to15 = df_merge2[(df_merge2['Period.Ending'] == '2012-12-31') | 
        (df_merge2['Period.Ending'] == '2013-12-31') |
        (df_merge2['Period.Ending'] == '2014-12-31') |
        (df_merge2['Period.Ending'] == '2015-12-31')]
df_ana12to15['Period.Ending'] = pd.DatetimeIndex(df_ana12to15['Period.Ending']).year

Items in the factor variables for which there are no entries in the data set must be deleted.

In [117]:
print(df_ana12to15['GICS.Sector'].cat.categories)

Index(['Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials',
       'Health Care', 'Industrials', 'Information Technology', 'Materials',
       'Real Estate', 'Telecommunications Services', 'Utilities'],
      dtype='object')


In [118]:
# Drop unused categories

df_ana12to15['GICS.Sector'] = df_ana12to15['GICS.Sector'].cat.remove_unused_categories()

In [119]:
print(df_ana12to15['GICS.Sector'].cat.categories)

Index(['Consumer Discretionary', 'Consumer Staples', 'Energy', 'Health Care',
       'Industrials', 'Information Technology', 'Materials', 'Real Estate',
       'Telecommunications Services', 'Utilities'],
      dtype='object')


### Defining additional variables

The variables enterprise value and EBITDA, which as value drivers are relevant for the analysis, are currently not available in the data set `df_ana12to15`, but can be generated by the other variables.

In [120]:
# Calculate EBITDA
df_ana12to15['EBITDA'] = df_ana12to15['Earnings.Before.Interest.and.Tax'] + df_ana12to15['Depreciation']
df_ana12to15['EBITDA']

2       5.441000e+09
3       6.983000e+09
4       6.507000e+09
5       3.546000e+09
6       8.167000e+09
            ...     
1341    5.910000e+08
1346    1.426100e+09
1354    1.012000e+09
1355    1.141000e+09
1356    8.680000e+08
Name: EBITDA, Length: 705, dtype: float64

In [121]:
# Calculate EV
df_ana12to15['EV'] = df_ana12to15['close'] * df_ana12to15['Estimated.Shares.Outstanding'] + df_ana12to15['Total.Liabilities'] - df_ana12to15['Cash.and.Cash.Equivalents']
df_ana12to15['EV']

2       7.788417e+10
3       6.999030e+10
4       9.960700e+10
5       1.220091e+11
6       1.374459e+11
            ...     
1341    8.494064e+09
1346    1.391727e+10
1354    2.132063e+10
1355    2.602128e+10
1356    2.958053e+10
Name: EV, Length: 705, dtype: float64

### Calculating multiples using location parameters

Create a subset of `df_ana12to15` called `df_HC` that contains only observations from the `Health Care` sector for the years 2012 to 2014.

In [122]:
df_HC = df_ana12to15[df_ana12to15['GICS.Sector'] == 'Health Care']
df_HC

,Ticker.Symbol,Period.Ending,Accounts.Payable,Accounts.Receivable,Add'l.income/expense.items,After.Tax.ROE,Capital.Expenditures,Capital.Surplus,Cash.Ratio,Cash.and.Cash.Equivalents,...,volume,Security,SEC.filings,GICS.Sector,GICS.Sub.Industry,Address.of.Headquarters,Date.first.added,CIK,EBITDA,EV
4,ABBV,2013,6.448000e+09,6.810000e+08,-54000000.0,92.0,-4.910000e+08,3.671000e+09,144.0,9.595000e+09,...,3019700.0,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,6.507000e+09,9.960700e+10
5,ABBV,2014,6.954000e+09,-1.720000e+08,-651000000.0,102.0,-6.120000e+08,4.194000e+09,74.0,8.348000e+09,...,3917300.0,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,3.546000e+09,1.220091e+11
6,ABBV,2015,8.463000e+09,-1.076000e+09,-206000000.0,130.0,-5.320000e+08,1.308000e+10,77.0,8.399000e+09,...,6019300.0,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,8.167000e+09,1.374459e+11
12,ABT,2013,5.948000e+09,-1.130000e+08,53000000.0,10.0,-1.145000e+09,0.000000e+00,85.0,3.475000e+09,...,4594200.0,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,3.905000e+09,7.451315e+10
13,ABT,2014,5.350000e+09,-1.950000e+08,69000000.0,11.0,-1.077000e+09,0.000000e+00,43.0,4.063000e+09,...,3418600.0,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,4.216000e+09,8.416845e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,XRAY,2015,4.639000e+08,-9.000000e+05,10400000.0,11.0,-7.200000e+07,2.378000e+08,60.0,2.846000e+08,...,417900.0,Dentsply Sirona,reports,Health Care,Health Care Supplies,"York, Pennsylvania",2008-11-14,818479,5.198000e+08,1.031970e+10
1346,ZBH,2012,2.069000e+08,-4.550000e+07,15600000.0,13.0,-2.636000e+08,3.500600e+09,180.0,8.843000e+08,...,1107900.0,Zimmer Biomet Holdings,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",NaT,1136869,1.426100e+09,1.391727e+10
1354,ZTS,2013,1.381000e+09,-9.900000e+07,9000000.0,54.0,-1.840000e+08,8.780000e+08,43.0,6.100000e+08,...,2270400.0,Zoetis,reports,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21,1555280,1.012000e+09,2.132063e+10
1355,ZTS,2014,1.071000e+09,6.900000e+07,-7000000.0,44.0,-1.800000e+08,9.580000e+08,81.0,8.820000e+08,...,2131900.0,Zoetis,reports,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21,1555280,1.141000e+09,2.602128e+10


Determine the multiple for `df_HC` based on the three location parameters: (1) Arithmetic mean (2) Median (3) Harmonic mean.

In [123]:
# Determine the EV/EBITDA ratio for each company

df_HC['EV/EBITDA.ratio'] = df_HC['EV'] / df_HC['EBITDA']
df_HC['EV/EBITDA.ratio']

4       15.307669
5       34.407528
6       16.829419
12      19.081472
13      19.964054
          ...    
1334    19.853206
1346     9.758972
1354    21.067819
1355    22.805683
1356    34.078950
Name: EV/EBITDA.ratio, Length: 93, dtype: float64

In [124]:
arithmetic_mean = np.mean(df_HC['EV/EBITDA.ratio'])
median_multiple = np.median(df_HC['EV/EBITDA.ratio'])
harmonic_mean = statistics.harmonic_mean(df_HC['EV/EBITDA.ratio'])

Then, calculate the company values on the basis of the resulting multiples.

In [125]:
df_HC['EV.mean'] = df_HC['EBITDA'] * arithmetic_mean
df_HC['EV.median'] = df_HC['EBITDA'] * median_multiple
df_HC['EV.harmonic.mean'] = df_HC['EBITDA'] * harmonic_mean

In [126]:
Company = ["A", "B", "C", "D", "E", "G", "H", "J", "K"]

EV = [20627, 18093, 23029, 13626, 27635, 11626, 16430, 23626, 25324]

EBITDA = [1534, 1322, 1863, 1245, 2109, 1075, 1408, 1900, 2033]

ex2 = pd.DataFrame(list(zip(Company, EV, EBITDA)), columns=["company", "EV", "EBITDA"])
ex2

,company,EV,EBITDA
0,A,20627,1534
1,B,18093,1322
2,C,23029,1863
3,D,13626,1245
4,E,27635,2109
5,G,11626,1075
6,H,16430,1408
7,J,23626,1900
8,K,25324,2033


In [127]:
ex2["ratio"] = ex2["EV"] / ex2["EBITDA"]
ex2

,company,EV,EBITDA,ratio
0,A,20627,1534,13.446545
1,B,18093,1322,13.686082
2,C,23029,1863,12.361245
3,D,13626,1245,10.944578
4,E,27635,2109,13.103367
5,G,11626,1075,10.814884
6,H,16430,1408,11.669034
7,J,23626,1900,12.434737
8,K,25324,2033,12.456468


In [128]:
mean = np.mean(ex2["ratio"])
mean

12.324104415913801

In [129]:
EV_F = mean * 1289
EV_F

15885.770592112889

In [130]:
median = np.median(ex2["ratio"])
median

12.434736842105263

In [131]:
EV_F2 = np.round(median * 1289, 2)
EV_F2

16028.38

In [132]:
harm_mean = statistics.harmonic_mean(ex2["ratio"])
harm_mean

12.246885855334346

In [133]:
EV_F3 = harm_mean * 1289
EV_F3

15786.235867525973

## Modeling

Applying SLR: we estimate a simple linear regression model based on the appropriately processed data set `df_ana12to15`.

With the function `sm.OLS` a regression model can be estimated. The dependent variable is stored in `df_ana12to15['EV']` , the independent variable in `df_ana12to15['EBITDA']`.

In [134]:
# Running the first regression

reg1 = sm.OLS(df_ana12to15['EV'], sm.add_constant(df_ana12to15['EBITDA'])).fit()
reg1.params

const     1.772406e+10
EBITDA    8.238525e+00
dtype: float64

Meaning that an increase in EBITDA (x) of 1 million USD leads on average to an increase in the estimated enterprise value EV (y) of 8.238 x 10^6 USD.

In [135]:
import plotly.express as px

fig = px.scatter(df_ana12to15, x='EBITDA', y='EV', opacity=0.65, trendline='ols', trendline_color_override='darkblue')
fig

In [136]:
import plotly.figure_factory as ff

hist_data = [df_ana12to15['EV'].to_numpy()]
group_labels = ['EV'] # name of the dataset

fig2 = ff.create_distplot(hist_data, group_labels, bin_size=1000000000)
fig2.show()

In [137]:
hist_data = [df_ana12to15['EBITDA'].to_numpy()]
group_labels = ['EBITDA'] # name of the dataset

fig3 = ff.create_distplot(hist_data, group_labels, bin_size=1000000000)
fig3.show()

The figure clearly shows that `EBITDA` and `EV` are right-skewed. Logarithmizing the variables is one option to compensate for the right-skewness. Further, logarithmising is used to stabilize the variance of variables.

In [138]:
# Logarithmic transformation of EBITDA and EV
df_ana12to15['lgEBITDA'] = np.log(df_ana12to15['EBITDA'])
df_ana12to15['lgEV'] = np.log(df_ana12to15['EV'])

In [139]:
hist_data = [df_ana12to15['lgEV'].to_numpy()]
group_labels = ['lgEV'] # name of the dataset

fig4 = ff.create_distplot(hist_data, group_labels, bin_size=1000000000)
fig4.show()

In [140]:
hist_data = [df_ana12to15['lgEBITDA'].to_numpy()]
group_labels = ['lgEBITDA'] # name of the dataset

fig5 = ff.create_distplot(hist_data, group_labels, bin_size=1000000000)
fig5.show()

In [141]:
# Running the log-log regression

df_ana12to15_OLS = df_ana12to15[['lgEV', 'lgEBITDA']]
reg2 = sm.OLS(df_ana12to15['lgEV'], sm.add_constant(df_ana12to15['lgEBITDA'])).fit()
reg2.params

const       5.709051
lgEBITDA    0.857239
dtype: float64

Meaning that a 1% increase in `EBITDA` leads on average to an increase in `EV` of about 0.86% (hat_beta_1=0.8572392).

In [142]:
fig6 = px.scatter(df_ana12to15, x='lgEBITDA', y='lgEV', opacity=0.65, trendline='ols', trendline_color_override='darkblue')
fig6

### Estimating models employing various value drivers

In [143]:
# a level-level-model with EBITDA as the value driver

reg_HC_EBITDA = sm.OLS(df_HC['EV'], sm.add_constant(df_HC['EBITDA'])).fit()
reg_HC_EBITDA.params

const     1.625714e+10
EBITDA    1.154681e+01
dtype: float64

In [144]:
# a level-level-model with EBIT as the value driver

reg_HC_EBIT = sm.OLS(df_HC['EV'], sm.add_constant(df_HC['Earnings.Before.Interest.and.Tax'])).fit()
reg_HC_EBIT.params

const                               2.126442e+10
Earnings.Before.Interest.and.Tax    1.404347e+01
dtype: float64

A subset of `df_ana12to15` containing only observations from the `Materials` sector for the years 2012 and 2013

In [145]:
df_Mat = df_ana12to15[df_ana12to15['GICS.Sector'] == 'Materials']

In [146]:
df_Mat_ana12to13 = df_Mat[(df_Mat['Period.Ending'] == 2012) | (df_Mat['Period.Ending'] == 2013)]

In [147]:
df_Mat_ana12to13

,Ticker.Symbol,Period.Ending,Accounts.Payable,Accounts.Receivable,Add'l.income/expense.items,After.Tax.ROE,Capital.Expenditures,Capital.Surplus,Cash.Ratio,Cash.and.Cash.Equivalents,...,SEC.filings,GICS.Sector,GICS.Sub.Industry,Address.of.Headquarters,Date.first.added,CIK,EBITDA,EV,lgEBITDA,lgEV
59,ALB,2012,3.723090e+08,-25992000.0,1229000.0,17.0,-2.808730e+08,2.761000e+06,124.0,4.776960e+08,...,reports,Materials,Specialty Chemicals,"Baton Rouge, Louisiana",2016-07-01,915913,5.000320e+08,6.671159e+09,20.030183,22.621059
60,ALB,2013,4.118090e+08,-65906000.0,-6674000.0,25.0,-1.553460e+08,9.957000e+06,109.0,4.772390e+08,...,reports,Materials,Specialty Chemicals,"Baton Rouge, Louisiana",2016-07-01,915913,6.773710e+08,6.792755e+09,20.333730,22.639122
180,BLL,2012,1.225300e+09,600000.0,-15100000.0,36.0,-3.050000e+08,0.000000e+00,10.0,1.741000e+08,...,reports,Materials,Metal & Glass Containers,"Broomfield, Colorado",1984-10-31,9389,1.058300e+09,1.301320e+10,20.779930,23.289230
181,BLL,2013,1.240100e+09,80200000.0,-28000000.0,33.0,-3.783000e+08,0.000000e+00,22.0,4.160000e+08,...,reports,Materials,Metal & Glass Containers,"Broomfield, Colorado",1984-10-31,9389,1.067300e+09,1.371216e+10,20.788398,23.341549
226,CF,2012,9.396000e+08,53200000.0,5400000.0,32.0,-5.235000e+08,2.492400e+09,239.0,2.274900e+09,...,reports,Materials,Fertilizers & Agricultural Chemicals,"Deerfield, Illinois",2008-08-27,1324404,3.384600e+09,4.651379e+09,21.942502,22.260429
337,DD,2013,1.164600e+10,-976000000.0,259000000.0,30.0,-1.882000e+09,1.107200e+10,68.0,8.941000e+09,...,reports,Materials,Diversified Chemicals,"Wilmington, Delaware",NaT,30554,4.617000e+09,8.363190e+10,22.253011,25.149691
403,ECL,2012,1.475900e+09,-189700000.0,0.0,12.0,-5.745000e+08,4.249100e+09,38.0,1.157800e+09,...,reports,Materials,Specialty Chemicals,"St. Paul, Minnesota",1989-01-31,31462,2.003800e+09,3.132872e+10,21.418311,24.167801
404,ECL,2013,1.673900e+09,-147400000.0,0.0,13.0,-6.251000e+08,4.692000e+09,10.0,3.392000e+08,...,reports,Materials,Specialty Chemicals,"St. Paul, Minnesota",1989-01-31,31462,2.376800e+09,4.319526e+10,21.589021,24.488997
421,EMN,2012,1.360000e+09,48000000.0,-8000000.0,15.0,-4.650000e+08,1.709000e+09,18.0,2.490000e+08,...,reports,Materials,Diversified Chemicals,"Kingsport, Tennessee",1994-01-01,915389,1.152000e+09,1.843062e+10,20.864765,23.637279
494,FCX,2012,3.100000e+09,-365000000.0,-82000000.0,18.0,-3.494000e+09,1.911900e+10,111.0,3.705000e+09,...,reports,Materials,Copper,"Phoenix, Arizona",NaT,831259,7.101000e+09,4.732325e+10,22.683501,24.580268


In [148]:
# a log-log model with EBITDA as the underlying value driver

df_Mat_ana12to13['lgEV'] = np.log(df_Mat_ana12to13['EV'])
df_Mat_ana12to13['lgEBITDA'] = np.log(df_Mat_ana12to13['EBITDA'])

reg3 = sm.OLS(df_Mat_ana12to13['lgEV'], sm.add_constant(df_Mat_ana12to13['lgEBITDA'])).fit()
reg3.params

const       7.911511
lgEBITDA    0.746086
dtype: float64

## Data Preparation II

The data must be further prepared so can control for time- and sector-specific effects.

### Balancing the data

Via the commands `groupby()` and `unstack()` the number of observations per year and sector is printed in tabular form.

In [149]:
tab = df_ana12to15.groupby(['GICS.Sector', 'Period.Ending']).size()
tab.unstack()

/var/folders/_s/mrzypwnj2sb6kyhjchmxrpmc0000gn/T/ipykernel_80584/417363599.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Period.Ending,2012,2013,2014,2015
GICS.Sector,,,,
Consumer Discretionary,13,21,20,20
Consumer Staples,6,11,11,11
Energy,17,25,20,9
Health Care,17,25,26,25
Industrials,18,42,47,45
Information Technology,9,17,17,17
Materials,14,14,18,16
Real Estate,13,13,15,16
Telecommunications Services,4,3,5,4


The table reveals that the number of observations differs over the years (unbalanced panel). `df_ana12to15` is then reduced to those enterprises for which observations are available for all four years (balanced panel), and a column is added to the above table.

In [167]:
# For the purpose of simplicity focus on balanced data

df2 = df_ana12to15
df2['Ticker.Symbol'] = df2['Ticker.Symbol'].astype('category')

tab2 = df2.groupby(['Ticker.Symbol']).size()
indices = (tab2==4) # we are only interested in the indices that occur in every year (4 times)
df_panel = df_ana12to15[df_ana12to15['Ticker.Symbol'].isin(tab2.loc[indices].index.values)]

gics = df_panel[df_panel['Period.Ending']==2012]
gics.groupby(['GICS.Sector']).size()

/var/folders/_s/mrzypwnj2sb6kyhjchmxrpmc0000gn/T/ipykernel_80584/4224269171.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/var/folders/_s/mrzypwnj2sb6kyhjchmxrpmc0000gn/T/ipykernel_80584/4224269171.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



GICS.Sector
Consumer Discretionary          8
Consumer Staples                6
Energy                          5
Health Care                    14
Industrials                    15
Information Technology          9
Materials                       9
Real Estate                     9
Telecommunications Services     3
Utilities                      15
dtype: int64

### Divide into training and test data

In [151]:
# Split Data into test and Train dataset

df_test = df_panel[df_panel['Period.Ending']==2015]
df_train = df_panel[df_panel['Period.Ending'].isin([2012,2013,2014])]

Define a subset of `df_ana12to15_balanced` called `df_HC_balanced`, which only contains observations from the `Health Care` sector.

In [152]:
df_ana12to15_balanced = df_panel
sec0 = "Health Care"

df_HC_balanced = df_ana12to15_balanced[df_ana12to15_balanced['GICS.Sector'].isin([sec0])]

Let's divide `df_HC_balanced` into a training set `df_HC_train` (observation period 2012 until 2014) and a test set `df_HC_test` (observation period 2015)

In [153]:
df_HC_train = df_HC_balanced[df_HC_balanced['Period.Ending'].isin([2012,2013,2014])]
df_HC_test = df_HC_balanced[df_HC_balanced['Period.Ending']==2015]

Now we run a regression according to the regression objects `reg_HC_EBITDA` and `reg_HC_EBIT` on the newly created data sets.

In [169]:
reg_HC_EBITDA_update = sm.OLS(df_HC_train['EV'], sm.add_constant(df_HC_train['EBITDA'])).fit()
reg_HC_EBITDA_update.params

const     1.024927e+10
EBITDA    1.180073e+01
dtype: float64

In [195]:
reg_HC_EBIT_update = sm.OLS(df_HC_train['EV'], sm.add_constant(df_HC_train['Earnings.Before.Interest.and.Tax'])).fit()
reg_HC_EBIT_update.params

const                               1.243105e+10
Earnings.Before.Interest.and.Tax    1.589983e+01
dtype: float64

### Select two sectors

In [155]:
# Industrial and Information Technology are supposed to behave different

sec1 = "Industrials"
sec2 = "Information Technology"

## Modeling II

In [156]:
# Build subset for (Mod1) – pooled regression: do not differ for time or sectors
dat_s1s2 = df_train[df_train['GICS.Sector'].isin([sec1,sec2])]
dat_s1s2_OLS = dat_s1s2[['lgEV', 'lgEBITDA']] # a smaller data set as a helper

# Build subset for (Mod2): only time differ
dat_s1s2_12 = df_train[(df_train['Period.Ending'] == 2012) & (df_train['GICS.Sector'].isin([sec1,sec2]))]
dat_s1s2_12_OLS = dat_s1s2_12[['lgEV', 'lgEBITDA']]
dat_s1s2_13 = df_train[(df_train['Period.Ending'] == 2013) & (df_train['GICS.Sector'].isin([sec1,sec2]))]
dat_s1s2_13_OLS = dat_s1s2_13[['lgEV', 'lgEBITDA']]
dat_s1s2_14 = df_train[(df_train['Period.Ending'] == 2014) & (df_train['GICS.Sector'].isin([sec1,sec2]))]
dat_s1s2_14_OLS = dat_s1s2_14[['lgEV', 'lgEBITDA']]

# Build subset for (Mod3): only sectors differ
dat_s1 = df_train[(df_train['GICS.Sector']==sec1)]
dat_s1_OLS = dat_s1[['lgEV', 'lgEBITDA']]

dat_s2 = df_train[(df_train['GICS.Sector']==sec2)]
dat_s2_OLS = dat_s2[['lgEV', 'lgEBITDA']]

# Build subset for (Mod4): both time and sector differ
dat_s1_12 = df_train[(df_train['Period.Ending']==2012) & (df_train['GICS.Sector']==sec1)]
dat_s1_12_OLS = dat_s1_12[['lgEV', 'lgEBITDA']]

dat_s1_13 = df_train[(df_train['Period.Ending']==2013) & (df_train['GICS.Sector']==sec1)]
dat_s1_13_OLS = dat_s1_13[['lgEV', 'lgEBITDA']]

dat_s1_14 = df_train[(df_train['Period.Ending']==2014) & (df_train['GICS.Sector']==sec1)]
dat_s1_14_OLS = dat_s1_14[['lgEV', 'lgEBITDA']]

dat_s2_12 = df_train[(df_train['Period.Ending']==2012) & (df_train['GICS.Sector']==sec2)]
dat_s2_12_OLS = dat_s2_12[['lgEV', 'lgEBITDA']]

dat_s2_13 = df_train[(df_train['Period.Ending']==2013) & (df_train['GICS.Sector']==sec2)]
dat_s2_13_OLS = dat_s2_13[['lgEV', 'lgEBITDA']]

dat_s2_14 = df_train[(df_train['Period.Ending']==2014) & (df_train['GICS.Sector']==sec2)]
dat_s2_14_OLS = dat_s2_14[['lgEV', 'lgEBITDA']]

Now the log-log regressions are applied to the subsets.

In [157]:
# Estimating (Mod1)
reg_s1s2 = sm.OLS(dat_s1s2_OLS['lgEV'], sm.add_constant(dat_s1s2_OLS['lgEBITDA'])).fit()

# Estimating regression equations attributed to (Mod 2)
reg_s1s2_12 = sm.OLS(dat_s1s2_12_OLS['lgEV'], sm.add_constant(dat_s1s2_12_OLS['lgEBITDA'])).fit()
reg_s1s2_13 = sm.OLS(dat_s1s2_13_OLS['lgEV'], sm.add_constant(dat_s1s2_13_OLS['lgEBITDA'])).fit()
reg_s1s2_14 = sm.OLS(dat_s1s2_14_OLS['lgEV'], sm.add_constant(dat_s1s2_14_OLS['lgEBITDA'])).fit()

# Estimating regression equations attributed to (Mod 3)
reg_s1 = sm.OLS(dat_s1_OLS['lgEV'], sm.add_constant(dat_s1_OLS['lgEBITDA'])).fit()
reg_s2 = sm.OLS(dat_s2_OLS['lgEV'], sm.add_constant(dat_s2_OLS['lgEBITDA'])).fit()

# Estimating regression equations attributed to (Mod 4)
reg_s1_12 = sm.OLS(dat_s1_12_OLS['lgEV'], sm.add_constant(dat_s1_12_OLS['lgEBITDA'])).fit()
reg_s1_13 = sm.OLS(dat_s1_13_OLS['lgEV'], sm.add_constant(dat_s1_13_OLS['lgEBITDA'])).fit()
reg_s1_14 = sm.OLS(dat_s1_14_OLS['lgEV'], sm.add_constant(dat_s1_14_OLS['lgEBITDA'])).fit()
reg_s2_12 = sm.OLS(dat_s2_12_OLS['lgEV'], sm.add_constant(dat_s2_12_OLS['lgEBITDA'])).fit()
reg_s2_13 = sm.OLS(dat_s2_13_OLS['lgEV'], sm.add_constant(dat_s2_13_OLS['lgEBITDA'])).fit()
reg_s2_14 = sm.OLS(dat_s2_14_OLS['lgEV'], sm.add_constant(dat_s2_14_OLS['lgEBITDA'])).fit()

In [158]:
fig7 = px.scatter(dat_s1s2, x='lgEBITDA', y='lgEV', opacity=0.65, trendline='ols', trendline_color_override='darkblue', title="Regression line for Mod 1")
fig7

In [159]:
fig8 = px.scatter(dat_s1s2, x="lgEBITDA", y="lgEV", facet_col="Period.Ending", color="Period.Ending", trendline="ols", title="Regression line for 2")
fig8

In [160]:
fig9 = px.scatter(dat_s1s2, x="lgEBITDA", y="lgEV", facet_col="GICS.Sector", color="GICS.Sector", trendline="ols", title="Regression line for 3")
fig9

/usr/local/lib/python3.9/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [161]:
fig10 = px.scatter(dat_s1s2, x="lgEBITDA", y="lgEV", facet_col="Period.Ending", color="GICS.Sector", trendline="ols", title="Regression line for Mod 4")
fig10

/usr/local/lib/python3.9/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



The code below shows an example procedure for (Mod 2):

In [162]:
X_M2 = pd.concat([sm.add_constant(dat_s1s2_12['lgEBITDA']), sm.add_constant(dat_s1s2_13['lgEBITDA']), 
									sm.add_constant(dat_s1s2_14['lgEBITDA'])], axis = 1)
X_M2 = X_M2.fillna(0)
X_M2 = X_M2.sort_index()
X_M2

,const,lgEBITDA,const,lgEBITDA,const,lgEBITDA
23,1.0,20.945310,0.0,0.000000,0.0,0.000000
24,0.0,0.000000,1.0,21.064157,0.0,0.000000
25,0.0,0.000000,0.0,0.000000,1.0,21.093640
63,1.0,20.529681,0.0,0.000000,0.0,0.000000
64,0.0,0.000000,1.0,20.837487,0.0,0.000000
...,...,...,...,...,...,...
1300,0.0,0.000000,1.0,21.049183,0.0,0.000000
1301,0.0,0.000000,0.0,0.000000,1.0,21.071596
1335,1.0,21.672991,0.0,0.000000,0.0,0.000000
1336,0.0,0.000000,1.0,21.679162,0.0,0.000000


In [163]:
Y_M2 = pd.concat([dat_s1s2_12['lgEV'], dat_s1s2_13['lgEV'], dat_s1s2_14['lgEV']])
Y_M2 = Y_M2.sort_index()
Y_M2

23      23.575590
24      23.894537
25      24.216084
63      22.415774
64      22.563171
          ...    
1300    23.531134
1301    23.517393
1335    23.992966
1336    24.129076
1337    24.210371
Name: lgEV, Length: 72, dtype: float64

In [164]:
model = sm.OLS(Y_M2, X_M2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   lgEV   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     157.5
Date:                Sun, 10 Dec 2023   Prob (F-statistic):           2.78e-35
Time:                        20:05:41   Log-Likelihood:                -5.5322
No. Observations:                  72   AIC:                             23.06
Df Residuals:                      66   BIC:                             36.72
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9732      1.317      1.498      0.139      -0.656       4.602
lgEBITDA       1.0247      0.063     16.356      0.000       0.900       1.150
const          3.4540      1.249      2.765      0.007       0.960       5.948
lgEBITDA       0.9598      0.059     16.240      0.000       0.842       1.078
const          4.1047      1.285      3.194      0.002       1.539       6.670
lgEBITDA       0.9325      0.061     15.372      0.000       0.811       1.054
==============================================================================
Omnibus:                        8.522   Durbin-Watson:                   1.329
Prob(Omnibus):                  0.014   Jarque-Bera (JB):               17.087
Skew:                          -0.193   Prob(JB):                     0.000195
Kurtosis:                       5.355   Cond. No.                         502.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Seemingly unrelated regression (SUR)

In [165]:
# !pip install linearmodels
from linearmodels.system import SUR
from collections import OrderedDict

dat_sur = dat_s1s2.set_index('Period.Ending')

# Only leads to same result if an Intercept Column with 1.0 is added, to adjust interception.

dat_sur['Intercept'] = 1.0
mod_data = OrderedDict()
for year in ['2012','2013','2014' ]:
    year_data = dat_sur.loc[int(year)]
    dependent = year_data.lgEV
    exog = year_data[['Intercept','lgEBITDA']]
    mod_data[year] = {'dependent': dependent, 'exog': exog}


res = SUR(mod_data).fit(cov_type='unadjusted')
res

System GLS Estimation Summary                           
===================================================================================
Estimator:                        GLS   Overall R-squared:                   0.9198
No. Equations.:                     3   McElroy's R-squared:                 0.8753
No. Observations:                  24   Judge's (OLS) R-squared:             0.9198
Date:                Sun, Dec 10 2023   Berndt's R-squared:                  0.9274
Time:                        20:05:41   Dhrymes's R-squared:                 0.9198
                                        Cov. Estimator:                  unadjusted
                                        Num. Constraints:                      None
                   Equation: 2012, Dependent Variable: lgEV                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      2.6092     1.5138     1.7236     0.0848     -0.3578      5.5761
lgEBITDA       0.9944     0.0720     13.809     0.0000      0.8533      1.1355
                   Equation: 2013, Dependent Variable: lgEV                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      4.2808     1.1028     3.8817     0.0001      2.1193      6.4422
lgEBITDA       0.9206     0.0522     17.644     0.0000      0.8183      1.0229
                   Equation: 2014, Dependent Variable: lgEV                   
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      4.6883     0.8819     5.3159     0.0000      2.9598      6.4169
lgEBITDA       0.9050     0.0416     21.736     0.0000      0.8234      0.9866
==============================================================================

Covariance Estimator:
Homoskedastic (Unadjusted) Covariance (Debiased: False, GLS: True)
SystemResults, id: 0x131ec17f0

In [166]:
print(reg_s1s2_12.params)
print(reg_s1s2_13.params)
print(reg_s1s2_14.params)

const       1.973197
lgEBITDA    1.024672
dtype: float64
const       3.454001
lgEBITDA    0.959763
dtype: float64
const       4.104735
lgEBITDA    0.932539
dtype: float64


## Evaluation

In [171]:
# Calculate confidence interval manually 
import scipy.stats as stats

#The 5% confidence interval is already visible in the summary, however it will show how to calculate it manually
reg_s1s2_12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   lgEV   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     172.8
Date:                Sun, 10 Dec 2023   Prob (F-statistic):           6.77e-12
Time:                        20:13:29   Log-Likelihood:                -7.0885
No. Observations:                  24   AIC:                             18.18
Df Residuals:                      22   BIC:                             20.53
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9732      1.638      1.204      0.241      -1.425       5.371
lgEBITDA       1.0247      0.078     13.146      0.000       0.863       1.186
==============================================================================
Omnibus:                        3.022   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.221   Jarque-Bera (JB):                1.469
Skew:                           0.497   Prob(JB):                        0.480
Kurtosis:                       3.693   Cond. No.                         498.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [172]:
#Access the coefficient and standard error of our model
se = reg_s1s2_12.bse[1]
coefficient = reg_s1s2_12.params[1]

# Degrees of freedom.
# list is necessary to navigate via [0] through the OrderedDict
df = len(list(list(mod_data.values())[0].values())[0]) 

# Critical value for significance level 5 % of two-sided test
crit = stats.t.ppf(0.975,df, loc=0, scale=1)

# Lower and upper bound
[coefficient - crit * se, coefficient + crit * se]

/var/folders/_s/mrzypwnj2sb6kyhjchmxrpmc0000gn/T/ipykernel_80584/1788945821.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/_s/mrzypwnj2sb6kyhjchmxrpmc0000gn/T/ipykernel_80584/1788945821.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



[0.8637971744695357, 1.18554696126293]

In [173]:
#Alternativley or for comparison
reg_s1s2_12.conf_int(0.05)

,0,1
const,-1.424817,5.371211
lgEBITDA,0.863020,1.186324


As the confidence interval does not contain the zero, the null hypothesis H0:ß_1^1=0 can be rejected at a significance level of 5%.

### Parallels between F- and t-tests

The F-test’s statistic corresponds to the square of the t-test statistic, as shown by the following lines of code that test the null hypothesis:

In [204]:
# Display squared t-statistic

t_test = reg_s1s2_12.tvalues[1]
t_test

/var/folders/_s/mrzypwnj2sb6kyhjchmxrpmc0000gn/T/ipykernel_80584/1950806715.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



13.145738047481723

In [175]:
# Equivalent manual calculation
# Run Regression with intercept only
dat_s1s2_12_OLS_i = dat_s1s2_12_OLS
dat_s1s2_12_OLS_i = sm.add_constant(dat_s1s2_12_OLS_i['lgEBITDA'])
reg_s1s2_12_i = sm.OLS(dat_s1s2_12_OLS['lgEV'], dat_s1s2_12_OLS_i['const']).fit()

# Compute F-statistic
SSR_H0 = sum(reg_s1s2_12_i.resid**2)
SSR_H1 = sum(reg_s1s2_12.resid**2)
n = len(dat_s1s2_12['lgEV'])
F_test = (SSR_H0-SSR_H1)/(SSR_H1/(n-2))
F_test

172.8104288130076

In [179]:
# For Comparison

t_test ** 2

172.8104288130086

In [180]:
# Short Alternative in python for f Test

f_value = reg_s1s2_12.fvalue
f_value

172.81042881300766

### Chow-test as a special form of the F-test

In [181]:
### --ChowHypA------------
# squared sum of residuals of the three separate regressions
SSR_A_G1 = sum(reg_s1s2_12.resid**2)
SSR_A_G2 = sum(reg_s1s2_13.resid**2)
SSR_A_G3 = sum(reg_s1s2_14.resid**2)

# squared sum of residuals of the pooled regression (model under H_0 in hypothesis A)
SSR_H1 = sum(reg_s1s2.resid**2)

# number of total observations
n = len(dat_s1s2['lgEV'])
# number of regressors
p = 2 # we have one intercept and one slope parameter
# number of distinct groups
G = 3

# Chow test statistic
Chow_A = ((SSR_H1 - sum((SSR_A_G1, SSR_A_G2, SSR_A_G3)))/(G*p)) / (sum((SSR_A_G1, SSR_A_G2, SSR_A_G3))/(n-G*p))

# Test decision - can H0 be rejected on a 5% significance level?
Chow_A > stats.f.ppf(0.95, dfn = G*p, dfd = n-G*p)

False

In [184]:
### --ChowHypB -----------------
SSR_B_G1 = sum(reg_s1.resid**2)
SSR_B_G2 = sum(reg_s2.resid**2)

G = 2

Chow_B = ((SSR_H1 - sum((SSR_B_G1, SSR_B_G2)))/(G*p)) / (sum((SSR_B_G1, SSR_B_G2))/(n-G*p))

Chow_B > stats.f.ppf(0.95, dfn = G*p, dfd = n-G*p)

False

In [183]:
### --ChowHypC -----------------
SSR_C_G1 = sum(reg_s1_12.resid**2)
SSR_C_G2 = sum(reg_s1_13.resid**2)
SSR_C_G3 = sum(reg_s1_14.resid**2)
SSR_C_G4 = sum(reg_s2_12.resid**2)
SSR_C_G5 = sum(reg_s2_13.resid**2)
SSR_C_G6 = sum(reg_s2_14.resid**2)

G = 6

Chow_C = ((SSR_H1 - sum((SSR_C_G1, SSR_C_G2, SSR_C_G3, SSR_C_G4, SSR_C_G5, SSR_C_G6)))/(G*p)) / (sum((SSR_C_G1, SSR_C_G2, SSR_C_G3, SSR_C_G4, SSR_C_G5, SSR_C_G6))/(n-G*p))

Chow_C > stats.f.ppf(0.95, dfn = G*p, dfd = n-G*p)

False

In [185]:
## --ChowHypD---------------------
SSR_D_G1 = sum(reg_s1_12.resid**2)
SSR_D_G2 = sum(reg_s1_14.resid**2)

# model under H_0
dat_s1_12_14 = pd.concat([dat_s1_12_OLS, dat_s1_14_OLS])
reg_s1_12_14 = sm.OLS(dat_s1_12_14['lgEV'], sm.add_constant(dat_s1_12_14['lgEBITDA'])).fit()

# squared sum of residuals in the model under H_0 
SSR_H1 = sum(reg_s1_12_14.resid**2)

# number of total observations
n = len(dat_s1_12_14['lgEV'])
# number of regressors
p = 2
# number of distinct groups
G = 2

Chow_D = ((SSR_H1 - sum((SSR_D_G1, SSR_D_G2)))/(G*p)) / (sum((SSR_D_G1, SSR_D_G2))/(n-G*p))

Chow_D > stats.f.ppf(0.95, dfn = G*p, dfd = n-G*p)

False

After the Chow-tests have been applied to the data, none of the four hypotheses can be rejected at a significance level of 5%.

### Prediction

This section deals with the prediction of enterprise values and their performance. The code below shows for the pooled model `reg_s1s2` the computation of the predictions for the year 2015 and their deviations from the actually observed values.

In [186]:
## --- Prediction Mod1 --------------------------------
# Build subset for forecast
dat_s1s2_forecast = df_test[df_test['GICS.Sector'].isin([sec1,sec2])]

# Calculate forecasts based on (Mod 1)
Mod1_forecast = reg_s1s2.predict(sm.add_constant(dat_s1s2_forecast['lgEBITDA']))
Mod1_errors = dat_s1s2_forecast['lgEV'] - Mod1_forecast

In [187]:
Mod1_RMSE = np.sqrt(np.mean(Mod1_errors**2))
Mod1_RMSE

0.2347831561014157

In [188]:
Mod1_MAE  = np.mean(np.abs(Mod1_errors))
Mod1_MAE

0.19363534040481398

In [189]:
# Build subset for (Mod3)
dat_s1_forecast = df_test[(df_test['GICS.Sector']==sec1)]
dat_s2_forecast = df_test[(df_test['GICS.Sector']==sec2)]

Mod3_s1_forecast = reg_s1.predict(sm.add_constant(dat_s1_forecast['lgEBITDA']))
Mod3_s2_forecast = reg_s2.predict(sm.add_constant(dat_s2_forecast['lgEBITDA']))

Mod3_errors = (pd.concat([dat_s1_forecast, dat_s2_forecast])['lgEV']) - pd.concat([Mod3_s1_forecast, Mod3_s2_forecast])

Mod3_RMSE = np.sqrt(np.mean(Mod3_errors**2))
Mod3_RMSE

0.2649041470574737

In [190]:
Mod3_MAE  = np.mean(np.abs(Mod3_errors))
Mod3_MAE

0.2159945379923814

The results reveal that model (Mod 1) outperforms model (Mod 3) with respect to the derived metrics RMSE and MAE.

In [197]:
## --- Prediction reg_HC_EBITDA_update and reg_HC_EBIT_update based on the test data df_HC_test ---------------------------

Mod_HC_EBITDA_forecast = reg_HC_EBITDA_update.predict(sm.add_constant(df_HC_test['EBITDA']))
Mod_HC_EBITDA_errors = df_HC_test['EV'] - Mod_HC_EBITDA_forecast

Mod_HC_EBIT_forecast = reg_HC_EBIT_update.predict(sm.add_constant(df_HC_test['Earnings.Before.Interest.and.Tax']))
Mod_HC_EBIT_errors = df_HC_test['EV'] - Mod_HC_EBIT_forecast

In [200]:
Mod_HC_EBITDA_RMSE = np.sqrt(np.mean(Mod_HC_EBITDA_errors**2))
Mod_HC_EBITDA_RMSE

55051790377.43536

In [201]:
Mod_HC_EBIT_RMSE = np.sqrt(np.mean(Mod_HC_EBIT_errors**2))
Mod_HC_EBIT_RMSE

76511985410.03447

In [202]:
Mod_HC_EBITDA_MAE = np.mean(np.abs(Mod_HC_EBITDA_errors))
Mod_HC_EBITDA_MAE

36431898228.85294

In [203]:
Mod_HC_EBIT_MAE = np.mean(np.abs(Mod_HC_EBIT_errors))
Mod_HC_EBIT_MAE

50396765106.311516